In [ ]:
import os
import glob
import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
#os.chdir("/Users/quynh/Downloads/kapa_coverage")

In [ ]:
# import files
input_files = []
path = os.listdir('/Users/administrator/Desktop/BK-geneious/dvg-junctions/')
for file in path:
    if file.endswith(".csv"):
        input_files.append(file)    
input_files.sort()
print(input_files)
len(input_files)

In [ ]:
cd /Users/administrator/Desktop/BK-geneious/dvg-junctions/

In [ ]:
df1 = pd.read_csv(input_files[1], index_col=0)
df1 = df1.drop(["Length", "Reads supporting discovery"], axis=1)
df1 = df1.rename(columns={'Reads using':'Reads','Junction Type':'Type', "Rearrangement Distance":'Length'})
df1['Minimum'] = df1['Minimum'].str.replace(">","")
df1['Minimum'] = df1['Minimum'].str.replace("<","")
df1['Maximum'] = df1['Maximum'].str.replace(">","")
df1['Maximum'] = df1['Maximum'].str.replace("<","")
Name_lst = []
Start_lst = []
End_lst = []
Reads_lst = []
Reads_lst = []
Type_lst = []
Length_lst = []
df1 = df1.reset_index()

In [ ]:
# For deletion
df_deletion = df1.loc[df1.Type == 'Deletion','Name':]
Name_lst = df_deletion['Name'].drop_duplicates()
Name_lst = Name_lst.values.tolist()
Start_lst = df_deletion.Minimum.tolist()
End_lst = df_deletion.Maximum.tolist()
Reads_lst = df_deletion['Reads']
Reads_lst = Reads_lst.values.tolist()
Length_lst = df_deletion['Length']
Length_lst = Length_lst.values.tolist()
Type_lst = ['Deletion']*len(Name_lst)

# For rearrangement 
df_rep = df1.loc[df1.Type =='Rearrangement', 'Name':]

temp_location = df_rep.Minimum.tolist()
temp_length = df_rep.Length.tolist()
for i in range(0,len(temp_location)):
    if i % 2 == 1:
        End_lst.append(temp_location[i])
        Length_lst.append(temp_length[i])
    else:
        Start_lst.append(temp_location[i])

Re_Name_lst = df_rep['Name'].drop_duplicates()
Re_Name_lst = Re_Name_lst.values.tolist()
for i in Re_Name_lst:
    Name_lst.append(i)
    
Re_Reads_lst = df_rep['Reads']
Re_Reads_lst = Re_Reads_lst[Re_Reads_lst.index % 2 == 0]
Re_Reads_lst = Re_Reads_lst.values.tolist()
for i in Re_Reads_lst:
    Reads_lst.append(i)

Type_lst = Type_lst + ['Rearrangement']*len(Re_Name_lst)   

# For rearrangement (inversion)
df_rep_inv = df1.loc[df1.Type =='Rearrangement (inversion)', 'Name':]

temp_location_2 = df_rep_inv.Minimum.tolist()
temp_length_2 = df_rep_inv.Length.tolist()
for i in range(0,len(temp_location_2)):
    if i % 2 == 1:
        End_lst.append(temp_location_2[i])
        Length_lst.append(temp_length_2[i])
    else:
        Start_lst.append(temp_location_2[i])

Re_Name_lst = df_rep_inv['Name'].drop_duplicates()
Re_Name_lst = Re_Name_lst.values.tolist()
for i in Re_Name_lst:
    Name_lst.append(i)
    
Re_Reads_lst = df_rep_inv['Reads']
Re_Reads_lst = Re_Reads_lst[Re_Reads_lst.index % 2 == 0]
Re_Reads_lst = Re_Reads_lst.values.tolist()
for i in Re_Reads_lst:
    Reads_lst.append(i)

Type_lst = Type_lst + ['Rearrangement (inversion)']*len(Re_Name_lst)   

# Create length list
#Length_lst = []
#for i in range(0,len(Name_lst)):
#    length = abs(int(End_lst[i])-int(Start_lst[i]))
#    Length_lst.append(length)

print(len(Name_lst))
print(len(Start_lst))
print(len(End_lst))
print(len(Reads_lst))
print(len(Type_lst))
print(len(Length_lst))


In [ ]:
# create table
UBk_data = {'Name':Name_lst, 'Start':Start_lst, 'End':End_lst, 'Length':Length_lst, 'Reads':Reads_lst,'Type':Type_lst}
ubk_df = pd.DataFrame(UBk_data)
ubk_df = ubk_df.sort_values(by=['Reads'], ascending=False)
totalreads = ubk_df['Reads'].sum()
print('total reads =',totalreads)
ubk_df


In [ ]:
ubk_df.to_excel('UBK007.xlsx', index=False)

In [ ]:
# merge table
extension2 = 'xlsx'
all_files2 = [i for i in glob.glob('*.{}'.format(extension2))]
all_files2


In [ ]:
writer = pd.ExcelWriter('kapa-junctions.xlsx', engine='xlsxwriter')

for f in all_files2[0:25]:
    df = pd.read_excel(f, index_col='Name')
    df.to_excel(writer, sheet_name=os.path.basename(f))
    
writer.save()

## Add genes affected column

In [ ]:
df = pd.read_excel('kapa-junctions.xlsx')
df

In [ ]:
ori = list(range(97,119))
agno = list(range(388,588))
vp2 = list(range(624,1679))
vp3 = list(range(981,1679))
vp1 = list(range(1564,2652))
larget1 = list(range(2722,4566))
larget2 = list(range(4911,5153))
larget = larget1 + larget2
smallt = list(range(4635,5153))
start_lst = df.Start.tolist()
end_lst = df.End.tolist()
length = len(end_lst)
all_gene = ['VP1','VP2','VP3',"LargeT",'SmallT','Agnoprotein']
all_gene.sort()

In [ ]:
gene_affected = []
for i in range(0,length):
    gene_lst = []
    if start_lst[i] < end_lst[i]:
        lst = list(range(start_lst[i],end_lst[i]+1))
    if start_lst[i] > end_lst[i]:
        lst = list(range(start_lst[i],5154)) + list(range(0,end_lst[i]+1))
    for i in range(0,len(lst)):
        #if lst[i] in ori:
            #gene_lst.append("Ori")
        if lst[i] in agno:
            gene_lst.append("Agnoprotein")
        if lst[i] in vp1:
            gene_lst.append("VP1")
        if lst[i] in vp2:
            gene_lst.append('VP2')
        if lst[i] in vp3:
            gene_lst.append('VP3')
        if lst[i] in larget:
            gene_lst.append('LargeT')
        if lst[i] in smallt:
            gene_lst.append('SmallT')            
    gene_lst = list(set(gene_lst))
    gene_lst.sort()
    if gene_lst == all_gene:
        gene_lst = ["All"]
    if gene_lst == []:
        gene_lst = ['None']

    gene_affected.append(gene_lst)


In [ ]:
df['Gene Affected'] = gene_affected
df['Gene Affected'] = [', '.join(map(str, l)) for l in df['Gene Affected']]
df.head()

In [ ]:
df.to_excel('dvg-junctions-JC.xlsx', index=False)